In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img,img_to_array
def create_data_dict(inp_size=None):
    X,y=list(),list()
    feature_path='nn'
    label_path='SegmentationClass'
    path='/kaggle/input/pascalvoc/VOC2012'
    
    for file in os.listdir(os.path.join(path,label_path)):
        X.append(img_to_array(load_img(os.path.join(path,feature_path,file[:-4]+'.jpg'),target_size=inp_size)))
        y.append(img_to_array(load_img(os.path.join(path,label_path,file),target_size=inp_size)))
        
    return tf.convert_to_tensor(X),y
    
X,y=create_data_dict((128,128,3))
X=X/255
'''
with open('/kaggle/input/X.npy', 'wb') as f:
    np.save(f,X)
with open('/kaggle/input/y.npy', 'wb') as f:
    np.save(f,y)
'''
VOC_COLORMAP = [[0, 0, 0], [128, 0, 0], [0, 128, 0], [128, 128, 0],
                [0, 0, 128], [128, 0, 128], [0, 128, 128], [128, 128, 128],
                [64, 0, 0], [192, 0, 0], [64, 128, 0], [192, 128, 0],
                [64, 0, 128], [192, 0, 128], [64, 128, 128], [192, 128, 128],
                [0, 64, 0], [128, 64, 0], [0, 192, 0], [128, 192, 0],
                [0, 64, 128]]
VOC_CLASSES = ['background', 'aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair', 'cow',
               'diningtable', 'dog', 'horse', 'motorbike', 'person',
               'potted plant', 'sheep', 'sofa', 'train', 'tv/monitor']

def build_colormap2label():
    colormap2label=np.zeros(256**3)
    for i,color in enumerate(VOC_COLORMAP):
        idx=(color[0]*256+color[1])*256+color[2]
        colormap2label[idx]=i
    return colormap2label

def voc_label_indices(colormap,colormap2label):
    colormap=np.asarray(colormap,dtype=np.int32)
    idx=(colormap[:,:,0]*256+colormap[:,:,1])*256+colormap[:,:,2]
    return colormap2label[idx]

In [ ]:
colormap2label=build_colormap2label()
y=tf.convert_to_tensor([voc_label_indices(label,colormap2label) for label in y])

In [ ]:
from tensorflow.keras.models import Model,Sequential
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Conv2DTranspose,BatchNormalization,Activation,Add,Input,Dropout
from tensorflow.keras import regularizers
def resnet_block(n_filters,ip_model):
    
    bi=tf.keras.initializers.Constant(value=0.01)
    init=tf.keras.initializers.he_uniform()
    reg=None #regularizers.l1_l2(l1=1e-4, l2=1e-3)
    
    d=Conv2D(n_filters,(3,3),padding='same',kernel_initializer=init,bias_initializer=bi,kernel_regularizer=reg)(ip_model)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    
    d=Conv2D(n_filters,(3,3),padding='same',kernel_initializer=init,bias_initializer=bi,kernel_regularizer=reg)(d)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    
    d=Add()([d,ip_model])
    return d

def get_model(ip_shape,n_categories):
    
    bi=tf.keras.initializers.Constant(value=0.01)
    init=tf.keras.initializers.he_uniform()
    reg=None #regularizers.l1_l2(l1=1e-4, l2=1e-3)
    
    ip=Input(shape=ip_shape)
    d=Conv2D(64,(3,3),strides=2,padding='same',kernel_initializer=init,bias_initializer=bi,kernel_regularizer=reg)(ip)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    #d=Dropout(0.4)(d)
    
    d=Conv2D(128,(3,3),strides=2,padding='same',kernel_initializer=init,bias_initializer=bi,kernel_regularizer=reg)(d)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    #d=Dropout(0.4)(d)
    
    d=Conv2D(128,(3,3),strides=2,padding='same',kernel_initializer=init,bias_initializer=bi,kernel_regularizer=reg)(d)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    #d=Dropout(0.4)(d)
    
    d=Conv2D(128,(3,3),strides=2,padding='same',kernel_initializer=init,bias_initializer=bi,kernel_regularizer=reg)(d)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    #d=Dropout(0.4)(d)
    
    for i in range(10):
        d=resnet_block(128,d)
    #d=Dropout(0.4)(d)
    
    d=Conv2DTranspose(128,(3,3),strides=2,padding='same',kernel_initializer=init,bias_initializer=bi,kernel_regularizer=reg)(d)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    #d=Dropout(0.4)(d)
    
    d=Conv2DTranspose(128,(3,3),strides=2,padding='same',kernel_initializer=init,bias_initializer=bi,kernel_regularizer=reg)(d)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    #d=Dropout(0.4)(d)
    
    d=Conv2DTranspose(64,(3,3),strides=2,padding='same',kernel_initializer=init,bias_initializer=bi,kernel_regularizer=reg)(d)
    d=BatchNormalization()(d)
    d=Activation('relu')(d)
    #d=Dropout(0.4)(d)
    
    d=Conv2DTranspose(n_categories,(3,3),strides=2,padding='same',kernel_initializer=init,bias_initializer=bi,kernel_regularizer=reg)(d)
    d=BatchNormalization()(d)
    d=tf.keras.activations.softmax(d,axis=-1)
    
    seg=Model(ip,d)
    return seg
inp_shape,n_categories=(128,128,3),len(VOC_CLASSES)
n_categories

In [ ]:
from tensorflow.keras.optimizers import Adam
'''
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)nn

tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
with tpu_strategy.scope():
'''
lr_schedule=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.1,decay_steps=10000,decay_rate=0.9)
optimizer=Adam(learning_rate=lr_schedule)
model=get_model(inp_shape,n_categories)
sse=tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=Adam(lr=0.001),loss=sse,metrics=['accuracy'])

tpu_strategy.num_replicas_in_sync

In [ ]:
model.fit(X,y,validation_split=0.2,shuffle=True,epochs=1000,batch_size=128,verbose=2)

In [ ]:
model.save('/kaggle/working/segnet.h5')

In [ ]:
img=np.asarray(tf.squeeze(model.predict(tf.expand_dims(X[279],axis=0))))
img=np.argmax(img,axis=-1)
image=np.zeros((128,128,3))
for i in range(128):
    for j in range(128):
        image[i][j]=np.asarray(VOC_COLORMAP[img[i][j]])
plt.subplot(1,2,1)
plt.imshow(X[279])
plt.subplot(1,2,2)
plt.imshow(image)